# Instagram Crawler
crawling for text, comments, and photos 

## 0. ready
1) Make sure you install Selenium, BeautifulSoup, and ChromeDriver. <br>



## 1. import packages

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup as bs

import time
import sys
from imp import reload

import re

from urllib.request import urlopen
import requests
import datetime 
import wget
import html2text

import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

from random import *

## 2. login 

In [14]:
reload(sys)
#sys.setdefaultencoding('utf-8') python3에는 디폴트라 굳이 설정 ㄴㄴ

targetInstagramUrl = 'https://www.instagram.com/accounts/login/?source=auth_switcher'

options = webdriver.ChromeOptions()
#mobile_emulation = { "deviceName": "Nexus 5" }
#options.add_experimental_option("mobileEmulation", mobile_emulation)

login_id = input('put your login id : ')
login_pw = input('put your login password : ')

driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)
driver.get(targetInstagramUrl)

#login process ----------------------------------------------------------------- start
#login process ----------------------------------------------------------------- start
time.sleep(2)

driver.find_elements_by_name("username")[0].send_keys(login_id)
driver.find_elements_by_name("password")[0].send_keys(login_pw)

driver.find_element_by_xpath("//form/div[4]/button").submit()

time.sleep(3)

driver.find_element_by_xpath("/html/body/div[4]/div/div/div[3]/button[2]").click()
#login process ----------------------------------------------------------------- end
#login process ----------------------------------------------------------------- end

put your login id : dodozipsa
put your login password : effington13


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


## 3. search for hashtag

In [22]:
hashtag = input('What are you searching for? : ')
driver.get('https://www.instagram.com/explore/tags/'+ hashtag)

What are you searching for? : 클래스101


## 4. Get links of each posts

In [27]:
def get_post_url(count = 100) : 

    links = []
    last_height = driver.execute_script("return document.body.scrollHeight") # Get scroll height

    while len(links) < count :
        
        ## 1. find links in page sources
        source = driver.page_source
        data = bs(source, 'html.parser')
        body = data.find('body')
        script = body.find('div')
        
        for link in script.findAll('a'):
            if re.match("/p", link.get('href')):
                links.append('https://www.instagram.com'+link.get('href'))

        links = list(set(links))
        
        ## 2. Scroll down to bottom        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        ## 3. Wait to load page
        time.sleep(3)

        ## 4. Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        #time.sleep(uniform(1.0, 2.0))

        ## 4 - 1. if all pages are already scrolled : break.
        if new_height == last_height:
            break
            
        #elif len(links) == count : 
        #    break
            
        ## 4 - 2. else : going on
        else : 
            last_height = new_height
    
    return links

In [28]:
#test
a = get_post_url(500)

In [29]:
len(a)

504

In [30]:
a

['https://www.instagram.com/p/B7Io_hJguP4/',
 'https://www.instagram.com/p/B7GbTbBprlV/',
 'https://www.instagram.com/p/B7N0ATrD2aq/',
 'https://www.instagram.com/p/B7FtSqtlQKL/',
 'https://www.instagram.com/p/B7DaXZNH9QO/',
 'https://www.instagram.com/p/B7BjHYIl8mi/',
 'https://www.instagram.com/p/B7P-6LEBto4/',
 'https://www.instagram.com/p/B7FKsYAnq68/',
 'https://www.instagram.com/p/B7AfmHTJy4A/',
 'https://www.instagram.com/p/B67-RcXnEeo/',
 'https://www.instagram.com/p/B7BGY4xlobO/',
 'https://www.instagram.com/p/B7JAIVxgkST/',
 'https://www.instagram.com/p/B7IgTSJBv8Q/',
 'https://www.instagram.com/p/B7BwpsVDDxb/',
 'https://www.instagram.com/p/B69jQUYl4xN/',
 'https://www.instagram.com/p/B68QWCWpjlj/',
 'https://www.instagram.com/p/B7LuzStluHX/',
 'https://www.instagram.com/p/B7D_ubcDWOQ/',
 'https://www.instagram.com/p/B6-tMSuH5gi/',
 'https://www.instagram.com/p/B65Q-BhHguf/',
 'https://www.instagram.com/p/B7KfpaRjiWl/',
 'https://www.instagram.com/p/B6_IitOh7Uo/',
 'https://

## Get information from each link of link list

In [31]:
def info_from_post(link_list) : #get information from each link of link list
    result = pd.DataFrame()
    
    for i in range(len(link_list)) : 
        try:

            page = urlopen(link_list[i]).read()
            data = bs(page, 'html.parser')
            body = data.find('body')
            script = body.find('script')
            raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
            json_data=json.loads(raw)

            posts =json_data['entry_data']['PostPage'][0]['graphql']
            posts = json.dumps(posts)
            posts = json.loads(posts)

            x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
            x.columns =  x.columns.str.replace("shortcode_media.", "")
            result = result.append(x)

        except:
            np.nan
            
    result = result.drop_duplicates(subset = 'shortcode')
    result.index = range(len(result.index))
        
    return result


In [32]:
res = info_from_post(a)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [33]:
res.head()

,__typename,accessibility_caption,caption_is_edited,commenting_disabled_for_viewer,comments_disabled,dash_info.is_dash_eligible,dash_info.number_of_qualities,dash_info.video_dash_manifest,dimensions.height,dimensions.width,...,title,tracking_token,video_duration,video_url,video_view_count,viewer_can_reshare,viewer_has_liked,viewer_has_saved,viewer_has_saved_to_collection,viewer_in_photo_of_you
0,GraphImage,No photo description available.,False,False,False,NaN,NaN,NaN,1080,1080,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
1,GraphSidecar,NaN,True,False,False,NaN,NaN,NaN,1080,1080,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
2,GraphSidecar,NaN,False,True,False,NaN,NaN,NaN,1350,1080,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
3,GraphImage,No photo description available.,False,False,False,NaN,NaN,NaN,810,1080,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False
4,GraphSidecar,NaN,True,False,False,NaN,NaN,NaN,1080,1080,...,NaN,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,NaN,NaN,NaN,True,False,False,False,False


In [35]:
res.to_csv('savingname.csv', encoding = 'UTF-8-sig') #encoding for excel